# AFL Model - Part 4 - Weekly Predictions
Now that we have explored different algorithms for modelling, we can implement our chosen model and predict this week's AFL games! All you need to do is run the afl_modelling script each Thursday or Friday to predict the following week's games.

In [1]:
# Import Modules
from afl_feature_creation import prepare_afl_features
import afl_data_cleaning
import afl_feature_creation
import afl_modelling
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

## Creating The Features For This Weekend's Games
To actually predict this weekend's games, we need to create the same features that we have created in the previous tutorials for the games that will be played this weekend. This includes all the rolling averages, efficiency features, elo features etc. So the majority of this tutorial will be using previously defined functions to create features for the following weekend's games.

### Create Next Week's DataFrame
Let's first get our cleaned afl_data dataset, as well as the odds for next weekend and the 2018 fixture.

In [2]:
# Grab the cleaned AFL dataset and the column order
afl_data = afl_data_cleaning.prepare_afl_data()
ordered_cols = afl_data.columns

# Define a function which grabs the odds for each game for the following weekend
def get_next_week_odds(path):
    # Get next week's odds
    next_week_odds = pd.read_csv(path)
    next_week_odds = next_week_odds.rename(columns={"team_1": "home_team", 
                                                "team_2": "away_team", 
                                                "team_1_odds": "odds", 
                                                "team_2_odds": "odds_away"
                                               })
    return next_week_odds

# Import the fixture
# Define a function which gets the fixture and cleans it up
def get_fixture(path):
    # Get the afl fixture
    fixture = pd.read_csv(path)

    # Replace team names and reformat
    fixture = fixture.replace({'Brisbane Lions': 'Brisbane', 'Footscray': 'Western Bulldogs'})
    fixture['Date'] = pd.to_datetime(fixture['Date']).dt.date.astype(str)
    fixture = fixture.rename(columns={"Home.Team": "home_team", "Away.Team": "away_team"})
    return fixture

next_week_odds = get_next_week_odds("data/weekly_odds.csv")
fixture = get_fixture("data/afl_fixture_2018.csv")

In [3]:
fixture.tail()

,Date,Season,Season.Game,Round,home_team,away_team,Venue
211,2018-08-24,2018,212,NaN,Brisbane,West Coast,Gabba
212,2018-08-24,2018,213,NaN,Carlton,St Kilda,Etihad Stadium
213,2018-08-24,2018,214,NaN,Richmond,Melbourne,MCG
214,2018-08-24,2018,215,NaN,Port Adelaide,Essendon,Adelaide Oval
215,2018-08-24,2018,216,NaN,Fremantle,Collingwood,Optus Stadium


Now that we have these DataFrames, we will define a function which combines the fixture and next week's odds to create a single DataFrame for the games over the next 7 days. To use this function we will need Game IDs for next week. So we will create another function which creates Game IDs by using the Game ID from the last game played and adding 1 to it.

In [4]:
# Define a function which creates game IDs for this week's footy games
def create_next_weeks_game_ids(afl_data):
    odds = get_next_week_odds("data/weekly_odds.csv")

    # Get last week's Game ID
    last_afl_data_game = afl_data['Game'].iloc[-1]

    # Create Game IDs for next week
    game_ids = [(i+1) + last_afl_data_game for i in range(odds.shape[0])]
    return game_ids

# Define a function which creates this week's footy game DataFrame
def get_next_week_df(afl_data):
    # Get the fixture and the odds for next week's footy games
    fixture = get_fixture("data/afl_fixture_2018.csv")
    next_week_odds = get_next_week_odds("data/weekly_odds.csv")
    next_week_odds['Game'] = create_next_weeks_game_ids(afl_data)

    # Get today's date and next week's date and create a DataFrame for next week's games
    todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
    date_in_7_days = (datetime.datetime.today() + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    fixture = fixture[(fixture['Date'] >= todays_date) & (fixture['Date'] < date_in_7_days)].drop(columns=['Season', 'Season.Game', 'Round'])
    next_week_df = pd.merge(fixture, next_week_odds, on=['home_team', 'away_team'])

    # Split the DataFrame onto two rows for each game
    h_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds']]
    h_df['Team'] = h_df['home_team']
    h_df['Opposition'] = h_df['away_team']
    h_df['Home?'] = 1
    a_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds_away']].rename(columns={'odds_away': 'odds'})
    a_df['Team'] = a_df['away_team']
    a_df['Opposition'] = a_df['home_team']
    a_df['Home?'] = 0
    next_week = a_df.append(h_df).sort_values(by='Game')
    return next_week

In [5]:
next_week_df = get_next_week_df(afl_data)
game_ids_next_round = create_next_weeks_game_ids(afl_data)
next_week_df.head()

,Date,Game,home_team,away_team,odds,Team,Opposition,Home?
0,2018-08-10,15369,Essendon,St Kilda,5.60,St Kilda,Essendon,0
0,2018-08-10,15369,Essendon,St Kilda,1.16,Essendon,St Kilda,1
1,2018-08-11,15370,Hawthorn,Geelong,1.73,Geelong,Hawthorn,0
1,2018-08-11,15370,Hawthorn,Geelong,2.10,Hawthorn,Geelong,1
2,2018-08-11,15371,Gold Coast,Richmond,1.05,Richmond,Gold Coast,0


### Create Each Feature
Now let's append next week's DataFrame to our afl_data DataFrame and then create all the features we used in the [AFL Feature Creation Tutorial](0.2. afl_feature_creation_tutorial.ipynb).

In [6]:
# Append next week's games to our afl_data DataFrame
afl_data = afl_data.append(next_week_df).reset_index(drop=True)
afl_data = afl_data[ordered_cols]

# Create disposal efficiency column
afl_data['disposal_efficiency'] = afl_data['ED'] / afl_data['D']

# Create rolling averages for our statistics
cols_indx_start = afl_data.columns.get_loc("GA")
afl_avgs = afl_feature_creation.create_rolling_averages(afl_data, 6, afl_data.columns[cols_indx_start:])

# Create form between teams feature
afl_avgs = afl_feature_creation.form_between_teams(afl_avgs, 6)

# Apply elos
elos, probs, elo_dict = afl_feature_creation.elo_applier(afl_data, 24)
afl_avgs['home_elo'] = afl_avgs['Game'].map(elos).str[0]
afl_avgs['away_elo'] = afl_avgs['Game'].map(elos).str[1]

# Get elos for next week
afl_avgs.loc[afl_avgs['home_elo'].isna(), 'home_elo'] = afl_avgs[afl_avgs['home_elo'].isna()]['home_team'].map(elo_dict)
afl_avgs.loc[afl_avgs['away_elo'].isna(), 'away_elo'] = afl_avgs[afl_avgs['away_elo'].isna()]['away_team'].map(elo_dict)

# Create Adjusted Margin and then Average it over a 6 game window
afl_avgs = afl_feature_creation.map_elos(afl_avgs)
afl_avgs['Adj_elo_ave_margin'] = afl_avgs['Margin'] * afl_avgs['elo_Opp'] / afl_avgs['elo']
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Adj_elo_ave_margin'])

# Create average elo of opponents beaten/lost to feature
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='beaten')
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='lost')

# Create regular margin rolling average
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Margin'])
afl_avgs.tail()

,Team,odds,Date,home_team,away_team,Behinds,Game,Goals,Home?,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Venue,Season,Status,GA_ave_6,CP_ave_6,UP_ave_6,ED_ave_6,CM_ave_6,MI5_ave_6,One.Percenters_ave_6,BO_ave_6,K_ave_6,HB_ave_6,D_ave_6,M_ave_6,G_ave_6,B_ave_6,T_ave_6,HO_ave_6,I50_ave_6,CL_ave_6,CG_ave_6,R50_ave_6,FF_ave_6,FA_ave_6,AF_ave_6,SC_ave_6,CCL_ave_6,SCL_ave_6,SI_ave_6,MG_ave_6,TO_ave_6,ITC_ave_6,T5_ave_6,disposal_efficiency_ave_6,form_over_opposition_6,home_elo,away_elo,elo,elo_Opp,Adj_elo_ave_margin_ave_6,average_elo_opponents_beaten_6,average_elo_opponents_lost_6,Margin_ave_6
3013,Collingwood,1.26,2018-08-11,Collingwood,Brisbane,NaN,15374,NaN,1,Brisbane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.333333,147.333333,258.333333,302.000000,12.000000,11.666667,51.833333,9.000000,217.000000,185.666667,402.666667,97.000000,13.166667,8.833333,62.500000,42.166667,53.500000,36.833333,57.666667,36.666667,22.833333,19.666667,1657.166667,1745.833333,11.500000,25.333333,97.500000,5769.833333,74.833333,76.833333,8.500000,0.749317,5,1501.449238,1280.219510,1501.449238,1280.219510,13.894854,1406.255837,1610.672888,16.166667
3014,Western Bulldogs,4.00,2018-08-12,North Melbourne,Western Bulldogs,NaN,15375,NaN,0,North Melbourne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.333333,137.500000,245.666667,282.333333,7.500000,10.333333,53.166667,6.500000,203.166667,180.000000,383.166667,85.833333,10.000000,7.500000,64.666667,31.666667,51.666667,36.333333,53.333333,33.166667,19.833333,20.000000,1544.666667,1609.000000,9.666667,26.666667,76.500000,5215.000000,74.000000,69.500000,11.666667,0.735500,4,1472.910710,1436.609139,1436.609139,1472.910710,-31.463520,1454.985523,1542.441101,-29.000000
3015,North Melbourne,1.34,2018-08-12,North Melbourne,Western Bulldogs,NaN,15375,NaN,1,Western Bulldogs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.333333,154.833333,216.833333,259.333333,11.833333,11.333333,49.333333,4.000000,191.666667,175.500000,367.166667,77.166667,12.833333,7.666667,63.500000,44.833333,52.833333,37.833333,55.500000,34.666667,20.833333,18.000000,1507.833333,1634.833333,14.166667,23.666667,89.500000,5442.166667,71.333333,73.666667,13.666667,0.704759,2,1472.910710,1436.609139,1472.910710,1436.609139,7.981990,1570.161952,1510.560437,6.833333
3016,Sydney,3.40,2018-08-12,Melbourne,Sydney,NaN,15376,NaN,0,Melbourne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.666667,154.833333,217.000000,265.000000,11.666667,9.000000,44.166667,6.666667,203.166667,167.833333,371.000000,81.500000,10.833333,7.500000,64.333333,29.000000,47.500000,35.666667,58.833333,47.666667,20.500000,24.833333,1494.500000,1636.500000,10.333333,25.333333,70.666667,5804.333333,73.000000,70.166667,8.500000,0.713457,6,1509.390579,1630.736013,1630.736013,1509.390579,-8.221143,1499.940734,1501.884047,-9.000000
3017,Melbourne,1.42,2018-08-12,Melbourne,Sydney,NaN,15376,NaN,1,Sydney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.666667,168.500000,232.000000,279.166667,11.166667,12.000000,50.333333,2.833333,216.166667,183.666667,399.833333,84.500000,15.000000,10.666667,74.833333,54.333333,62.333333,43.166667,59.833333,30.500000,19.166667,20.833333,1680.333333,1714.500000,14.833333,28.333333,117.666667,6266.833333,75.166667,78.833333,14.500000,0.694229,0,1509.390579,1630.736013,1509.390579,1630.736013,34.258681,1490.159807,1633.148523,32.666667


Our DataFrame looks great! Although there are some NaNs, these are only in columns like 'Goals' which obviously don't have a value yet as the game hasn't been played.

Now we need to get our DataFrame on one line so that each row corresponds to one footy game. Let's use our previously defined function for this. Let's also drop the columns with NaNs which we don't need.

In [7]:
# Get each footy match on individual rows and drop irrelevant columns
one_line = afl_modelling.get_df_on_one_line(afl_avgs)

# Drop duplicate columns and unnecessary columns
cols_to_drop = ['Opposition Behinds', 'Goals', 'Behinds', 'Opposition Goals', 'Opposition Points', 'Points', 'Round', 'Venue', 'Season', 'Status',
               'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'elo', 'elo_Opp', 'Behinds_away',
               'Goals_away', 'home_elo_away', 'away_elo_away', 'elo_away', 'elo_Opp_away']
one_line = one_line.drop(columns=cols_to_drop)
# Drop all columns where home_elo or away_elo == 1500 exactly, as this is the first game played
one_line = one_line[(one_line['home_elo'] != 1500) & (one_line['away_elo'] != 1500)]

# Drop Na rows from calculating moving averages
one_line = one_line.dropna(axis=0)

Now let's create our differential_df and then filter out DataFrame to only include this weekend's game, based on the Game IDs we created earlier

In [8]:
diff_df = afl_modelling.get_diff_df(one_line)
diff_df = diff_df.drop(columns=['odds', 'odds_away']).select_dtypes(include=[np.number])
prediction_feature_set = diff_df[diff_df['Game'].isin(game_ids_next_round)].dropna(axis=1)

In [9]:
prediction_feature_set

,Game,home_elo,away_elo,CCL_ave_6_away,SCL_ave_6_away,SI_ave_6_away,MG_ave_6_away,TO_ave_6_away,ITC_ave_6_away,T5_ave_6_away,GA_ave_6_diff,CP_ave_6_diff,UP_ave_6_diff,ED_ave_6_diff,CM_ave_6_diff,MI5_ave_6_diff,One.Percenters_ave_6_diff,BO_ave_6_diff,K_ave_6_diff,HB_ave_6_diff,D_ave_6_diff,M_ave_6_diff,G_ave_6_diff,B_ave_6_diff,T_ave_6_diff,HO_ave_6_diff,I50_ave_6_diff,CL_ave_6_diff,CG_ave_6_diff,R50_ave_6_diff,FF_ave_6_diff,FA_ave_6_diff,AF_ave_6_diff,SC_ave_6_diff,disposal_efficiency_ave_6_diff,form_over_opposition_6_diff,Adj_elo_ave_margin_ave_6_diff,average_elo_opponents_beaten_6_diff,average_elo_opponents_lost_6_diff,Margin_ave_6_diff,implied_odds_prob,implied_odds_prob_away
1531,15369,1518.535623,1416.739182,12.333333,25.333333,95.333333,5441.333333,70.833333,66.166667,11.833333,2.833333,9.833333,-34.666667,-20.500000,4.333333,3.333333,3.666667,3.833333,14.000000,-33.500000,-19.500000,-1.666667,2.833333,1.500000,2.166667,6.000000,3.500000,2.000000,5.166667,3.000000,-2.166667,3.500000,-9.500000,65.833333,-0.015062,0,46.485619,193.659899,-58.544288,42.000000,0.862069,0.178571
1532,15370,1600.259805,1643.524765,14.000000,24.833333,98.166667,5696.500000,72.500000,67.500000,11.666667,1.000000,-4.666667,19.833333,26.333333,0.166667,3.000000,2.666667,5.500000,14.333333,4.500000,18.833333,8.666667,2.166667,1.000000,4.833333,6.500000,1.666667,-4.666667,-1.833333,-8.833333,-1.166667,-2.500000,124.166667,67.833333,0.034245,-2,21.202592,10.901723,-114.435695,22.666667,0.476190,0.578035
1533,15371,1294.817584,1624.637563,12.333333,20.000000,107.000000,6061.000000,72.166667,83.666667,11.166667,-4.166667,1.000000,-41.833333,-50.333333,-2.833333,-5.000000,2.333333,-1.833333,-11.500000,-25.833333,-37.333333,-17.000000,-6.000000,2.500000,14.666667,16.166667,-9.333333,4.333333,0.666667,4.000000,2.666667,-1.000000,-90.166667,-182.000000,-0.070537,-2,-51.448958,-125.685102,-130.766785,-50.166667,0.050000,0.952381
1534,15372,1563.944697,1591.711462,9.833333,23.666667,79.166667,5513.000000,71.500000,72.833333,9.333333,0.500000,15.666667,5.333333,-6.166667,-1.000000,1.500000,10.666667,4.833333,-2.333333,22.666667,20.333333,-10.166667,0.333333,0.166667,11.000000,0.500000,2.666667,3.666667,4.833333,1.166667,-1.166667,0.833333,50.833333,99.333333,-0.056332,-4,12.418313,-126.106419,-44.817660,14.000000,0.636943,0.362319
1535,15373,1613.612204,1620.104725,10.166667,28.166667,69.500000,5638.833333,71.666667,76.000000,13.166667,3.166667,0.000000,31.833333,34.500000,1.833333,2.333333,9.000000,5.000000,0.500000,24.500000,25.000000,7.500000,2.666667,1.000000,-11.000000,1.833333,4.500000,4.833333,4.166667,7.833333,2.166667,2.000000,44.000000,112.166667,0.047941,-4,24.344861,6.228059,78.200924,26.666667,0.645161,0.352113
1536,15374,1501.449238,1280.219510,12.166667,27.500000,101.333333,5755.000000,65.666667,63.833333,9.000000,-0.833333,7.333333,17.833333,16.500000,2.333333,-1.833333,-6.333333,4.166667,2.000000,19.666667,21.666667,-2.666667,0.666667,0.833333,5.833333,-4.166667,0.500000,-2.833333,2.000000,-3.666667,-0.500000,-0.500000,62.333333,55.166667,0.002706,4,15.610295,-27.481118,40.912099,15.166667,0.793651,0.208333
1537,15375,1472.910710,1436.609139,9.666667,26.666667,76.500000,5215.000000,74.000000,69.500000,11.666667,3.000000,17.333333,-28.833333,-23.000000,4.333333,1.000000,-3.833333,-2.500000,-11.500000,-4.500000,-16.000000,-8.666667,2.833333,0.166667,-1.166667,13.166667,1.166667,1.500000,2.166667,1.500000,1.000000,-2.000000,-36.833333,25.833333,-0.030741,-2,39.445510,115.176429,-31.880664,35.833333,0.746269,0.250000
1538,15376,1509.390579,1630.736013,10.333333,25.333333,70.666667,5804.333333,73.000000,70.166667,8.500000,4.000000,13.666667,15.000000,14.166667,-0.500000,3.000000,6.166667,-3.833333,13.000000,15.833333,28.833333,3.000000,4.166667,3.166667,10.500000,25.333333,14.833333,7.500000,1.000000,-17.166667,-1.333333,-4.000000,185.833333,78.000000,-0.019227,-6,42.479823,-9.780928,131.264476,41.666667,0.704225,0.294118


Great! We now have our feature DataFrame for this weekend's games. Let's now model this to create our predictions

## Creating Our Predictions
To create our predictions we will use similar code to what we used in our AFL Modelling Tutorial. We will need to train our model on all the data we have (up until last week's games), and then use our trained model to predict this week. Again, we will use Stacking with XGB, like we did in our AFL Modelling Tutorial.

In [10]:
afl = prepare_afl_features(window=6, k_factor=24).dropna().sort_values(by='Game')
afl = afl_modelling.get_df_on_one_line(afl)

# Drop columns which leak data and which we don't need
dropped_cols = ['Behinds', 'Goals', 'Opposition Behinds', 'Opposition Goals', 'Opposition Points', 'Points',
               'elo', 'elo_Opp', 'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'home_win_away',
               'home_elo_away', 'away_elo_away']
afl = afl.drop(columns=dropped_cols)
# Get a differential DataFrame - subtracting the away features from the home features
diff_df = afl_modelling.get_diff_df(afl)

In [11]:
# Hard code estimators from our last tutorial
all_estimators = [
    LogisticRegression(C=0.75, class_weight=None, dual=False, fit_intercept=True,
       intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
       penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
       verbose=0, warm_start=False),
    
    RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
     max_iter=None, normalize=False, random_state=None, solver='auto',
     tol=0.001),
    
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
         max_depth=15, max_features='sqrt', max_leaf_nodes=None,
         min_impurity_decrease=0.0, min_impurity_split=None,
         min_samples_leaf=1, min_samples_split=10,
         min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=-1,
         oob_score=False, random_state=5, verbose=0, warm_start=False),
    
    GaussianNB(priors=None),
    
    LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
           solver='svd', store_covariance=False, tol=0.0001)
]

# Hard code best cols
best_cols = ['home_elo',
'away_elo',
'GA_ave_6_diff',
'CP_ave_6_diff',
'UP_ave_6_diff',
'ED_ave_6_diff',
'CM_ave_6_diff',
'MI5_ave_6_diff',
'One.Percenters_ave_6_diff',
'BO_ave_6_diff',
'HB_ave_6_diff',
'M_ave_6_diff',
'G_ave_6_diff',
'T_ave_6_diff',
'HO_ave_6_diff',
'I50_ave_6_diff',
'CL_ave_6_diff',
'CG_ave_6_diff',
'R50_ave_6_diff',
'FF_ave_6_diff',
'FA_ave_6_diff',
'AF_ave_6_diff',
'SC_ave_6_diff',
'disposal_efficiency_ave_6_diff',
'R50_efficiency_ave_6_diff',
'I50_efficiency_ave_6_diff',
'Adj_elo_ave_margin_ave_6_diff',
'average_elo_opponents_beaten_6_diff',
'average_elo_opponents_lost_6_diff',
'Margin_ave_6_diff',
'implied_odds_prob',
'implied_odds_prob_away']

In [12]:
# Create our train sets
X = diff_df.drop(columns=['home_win']).select_dtypes(include=[np.number])
y = diff_df['home_win']

features = prediction_feature_set.columns

# Predict Next Round
preds_next_round = afl_modelling.implement_xgb_stacking(X[features].drop(columns='Game'), y, 
                                                        prediction_feature_set.drop(columns='Game'), all_estimators)

C:\Users\wardj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Now that we have our predictions, the final step is to put our predictions into a DataFrame so they're easy on the eye, and our predictions correspond to actual home and away teams rather than Game IDs. This is the final step.

In [13]:
preds_df = pd.DataFrame({
    "Game": prediction_feature_set['Game'],
    "Prediction (home_win)": preds_next_round
})

prediction_feature_set['home_odds'] = 1 / prediction_feature_set['implied_odds_prob']
prediction_feature_set['away_odds'] = 1 / prediction_feature_set['implied_odds_prob_away']

final_preds_df = pd.merge(preds_df, next_week_df[['home_team', 'away_team', 'Game']], on='Game').drop_duplicates()
final_preds_df = pd.merge(final_preds_df, prediction_feature_set[['home_odds', 'away_odds', 'home_elo', 'away_elo', 'Game']], on='Game')
final_preds_df['Predicted Winner'] = final_preds_df.apply(lambda x: x['home_team'] if x['Prediction (home_win)'] == 1 else x['away_team'], axis=1)

In [15]:
final_preds_df[['Game', 'home_team', 'away_team', 'Predicted Winner', 'home_odds', 'away_odds', 'home_elo', 'away_elo']]

,Game,home_team,away_team,Predicted Winner,home_odds,away_odds,home_elo,away_elo
0,15369,Essendon,St Kilda,Essendon,1.16,5.60,1518.535623,1416.739182
1,15370,Hawthorn,Geelong,Hawthorn,2.10,1.73,1600.259805,1643.524765
2,15371,Gold Coast,Richmond,Richmond,20.00,1.05,1294.817584,1624.637563
3,15372,Port Adelaide,West Coast,Port Adelaide,1.57,2.76,1563.944697,1591.711462
4,15373,GWS,Adelaide,GWS,1.55,2.84,1613.612204,1620.104725
5,15374,Collingwood,Brisbane,Collingwood,1.26,4.80,1501.449238,1280.219510
6,15375,North Melbourne,Western Bulldogs,North Melbourne,1.34,4.00,1472.910710,1436.609139
7,15376,Melbourne,Sydney,Melbourne,1.42,3.40,1509.390579,1630.736013


## Conclusion
Congratulations! You have created AFL predictions for this week. If you are beginner to this, don't be overwhelmed. The process gets easier each time you do it. And it is super rewarding. In future iterations we will update this tutorial to predict actual odds, and then integrate this with Betfair's API so that you can create an automated betting strategy using Machine Learning to create your predictions!